In [65]:
#%pip install torch torchvision torchaudio
import pandas as pd
from transformers import pipeline
import torch


NameError: name 'pl' is not defined

NameError: name 'lazy_df' is not defined

NameError: name 'lazy_df' is not defined

NameError: name 'filtered_df' is not defined

NameError: name 'apple_df' is not defined

### 0. Cleaning our data

Here I needed to properly clean the dataset we have before processing it

In [102]:
#First choose what dataset we want to clean.
#I want us to first work with Apple news dataset but we could work with any dataset (just replace the path)
df_to_clean = pd.read_csv("../data/apple_news.csv")
df_to_clean.head()

,Unnamed: 0,Date,Stock_Symbol,Article_title
0,0,2023-12-16 22:00:00,AAPL,My 6 Largest Portfolio Holdings Heading Into 2024 -- and the Important Investing Lesson I Learned From Each One
1,1,2023-12-16 22:00:00,AAPL,Brokers Suggest Investing in Apple (AAPL): Read This Before Placing a Bet
2,2,2023-12-16 21:00:00,AAPL,"Company News for Dec 19, 2023"
3,3,2023-12-16 21:00:00,AAPL,NVIDIA (NVDA) Up 243% YTD: Will It Carry Momentum in 2024?
4,4,2023-12-16 21:00:00,AAPL,"Pre-Market Most Active for Dec 19, 2023 : BMY, SQQQ, NIO, UBS, TQQQ, UBER, NVDA, AAPL, GOTU, CAN, TSLA, PLTR"



TODO :
-Remove unnamed index column
-Parse the Date column to datetime format.
-Incorporate sentiment analysis with Dis roberta to generate sentiment scores

In [104]:
# Drop the column 'Unnamed: 0' from the DataFrame 'df_to_clean' becuase it is not needed for analysis.
df_to_clean = df_to_clean.drop(columns=['Unnamed: 0'])
# Convert the 'Date' column to a datetime object
df_to_clean['Date'] = pd.to_datetime(df_to_clean['Date'])
#filter the DataFrame 'df_filtered' to only include rows where the 'Article_title' column contains the word 'Apple' or 'AAPL'.
df_filtered = df_to_clean[df_to_clean['Article_title'].str.contains(r'\b(Apple|AAPL)\b', case=False, na=False)]

KeyError: "['Unnamed: 0'] not found in axis"

In [69]:
#Finally, save the cleaned dataset
#If we work with another dataset, change the path
df_filtered.to_csv("../data/clean/apple_news_c.csv")
df_filtered

,Date,Stock_Symbol,Article_title
1,2023-12-16 22:00:00,AAPL,Brokers Suggest Investing in Apple (AAPL): Read This Before Placing a Bet
4,2023-12-16 21:00:00,AAPL,"Pre-Market Most Active for Dec 19, 2023 : BMY, SQQQ, NIO, UBS, TQQQ, UBER, NVDA, AAPL, GOTU, CAN, TSLA, PLTR"
6,2023-12-16 20:00:00,AAPL,AAPL Quantitative Stock Analysis
15,2023-12-16 04:00:00,AAPL,"After Hours Most Active for Dec 18, 2023 : PACB, AAPL, VTIP, EDAP, FTNT, AMZN, HPE, SKT, VZ, MRK, BVN, PFE"
16,2023-12-16 04:00:00,AAPL,"Technology Sector Update for 12/18/2023: PCT, ADBE, AAPL, EBIX"
...,...,...,...
9099,2020-03-10 00:00:00,AAPL,Peloton Shares Tick To Session Low As Hearing Report Apple Working On 'Guided Workout' Fitness App
9105,2020-03-10 00:00:00,AAPL,"UBS Maintains Buy on Apple, Lowers Price Target to $335"
9115,2020-05-31 00:00:00,AAPL,"Apple CEO Writes To Employees About George Floyd Death, Urges For 'Better, More Just World For Everyone'"
9135,2020-05-28 00:00:00,AAPL,"Tredje AP-fonden Buys Microsoft Corp, Amazon.com Inc, Apple Inc, Sells iShares U.S. ..."


### 1. News-data Preprocessing

#### Here we start our data processing work to convert get the sentiment

Assume that the dataset of the news is a constant flow of the news, then we try to construct a stochastic process on sentiment: use distrillroberta to generate sentiment(positive, neutral, negative) of the news headline and labelled the as (1, 0, -1) scores. Noted that the news are always released in two ways: one is that the news are released at midnight(or non-trading time) in batches, the others are released during trading time without batches(a flow-released news). For batch-released news, average the sentiment score.

THe formula for the stochastic process is be written as 

$$
I_t^s = \sum_{j} g(f(e_{jt}^{s})), 
g(x) = \begin{cases} 1, x = "Positive"\\
0, x = "Neutral"\\
-1, x = "Negative"\\
\end{cases},

f(x) = roBERTa(x),

$$

$f(x)$ is the pretrained sentiment classifier. $e_{jt}^{s}$ means the $j$-th embedding of the news headline at time $t$ for the stock $s$.

**Note**: time $t$ is not the natural time for trading, but it's proxy time interval between the new release, and it differs from stock to stock. Think of it as some sort of jump process.

In [70]:
cleaned_df = pd.read_csv("../data/clean/apple_news_c.csv")
cleaned_df.head(2)

,Unnamed: 0,Date,Stock_Symbol,Article_title
0,1,2023-12-16 22:00:00,AAPL,Brokers Suggest Investing in Apple (AAPL): Read This Before Placing a Bet
1,4,2023-12-16 21:00:00,AAPL,"Pre-Market Most Active for Dec 19, 2023 : BMY, SQQQ, NIO, UBS, TQQQ, UBER, NVDA, AAPL, GOTU, CAN, TSLA, PLTR"


In [105]:
#%pip install torch
"""
This script performs sentiment analysis on financial news articles using the PRE-TRAINED model we selected from Hugging Face.
The sentiment analysis model used is 'distilroberta-finetuned-financial-news-sentiment-analysis', which is fine-tuned on the financial_phrasebank dataset.

Functions:
    classify_sentiment(article: str) -> int:
        Classifies the sentiment of a given article as positive, negative, or neutral.
        Returns 1 for positive sentiment, -1 for negative sentiment, and 0 for neutral sentiment.

The script reads a DataFrame 'cleaned_df' containing financial news articles, applies sentiment analysis to the 'Article_title' column,
and saves the resulting DataFrame with sentiment scores to a CSV file.

Usage:
    Ensure that the required libraries are installed and the input DataFrame 'cleaned_df' is loaded.
    Run the script to perform sentiment analysis and save the results to a CSV file.
"""
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

# We will use distilRoberta-financial-sentiment a fine-tuned version of distilroberta-base on the financial_phrasebank dataset
tokenizer = AutoTokenizer.from_pretrained("mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")
model = AutoModelForSequenceClassification.from_pretrained("mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")
sentiment_pipeline = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

# Let's apply the discussed sentiment analysis
def classify_sentiment(article):
    sentiment_result = sentiment_pipeline(article)[0]
    label = sentiment_result['label']
    if label == "positive":
        return 1
    elif label == "negative":
        return -1
    else:
        return 0

# Now we need to merge the sentiment scores to the dataset (AND AFTER MERGE INDEX)
cleaned_df['Sentiment'] = cleaned_df['Article_title'].apply(classify_sentiment)
cleaned_df.to_csv("../data/processed/processed_news_data_with_sent.csv", index=False)
print("Sentiment classification completed and saved to processed_news_data_with_sent.csv.")


Device set to use mps:0


Sentiment classification completed and saved to processed_news_data_with_sent.csv.


In [63]:
#Here we merge the sentiment scores to the dataset
merged_df = pd.read_csv("../data/processed/processed_news_data_with_sent.csv")
merged_df.head(2)

,Unnamed: 0,Date,Stock_Symbol,Article_title,Sentiment
0,1,2023-12-16 22:00:00,AAPL,Brokers Suggest Investing in Apple (AAPL): Read This Before Placing a Bet,0
1,4,2023-12-16 21:00:00,AAPL,"Pre-Market Most Active for Dec 19, 2023 : BMY, SQQQ, NIO, UBS, TQQQ, UBER, NVDA, AAPL, GOTU, CAN, TSLA, PLTR",0


### 2. Price-data preprocessing

#### Now that we have our data set with the sentiments, we want to link it to our stock price

Discretize the tick-by-tick data to align with the news-data, by identifying the return of the price. When the batch news are released in non-trading time, assume that the traders in the market will react to the news as soon as the trading time starts, for flow-released news, assume that the market participants will react as soon as the information of the news is understood. There will be two hyperparameter: the time-lag between the new release and the market reaction, and the fixed timeframe that uses to calculate the return of the price. Set the return of the stock as positive, stable and negative under a threshold and label it as (1,0,-1). Then we construct a stochastic process for stock return.

The formula for the stochastic process of the discretized return can be written as 

$$
R_{t}^{s} = h(\log \frac{P_{t + \delta}^{s}}{P_{t}^{s}}), 

h(x) = \begin{cases}1, x > \gamma \\ 0, \|x\| \leq \gamma \\ -1, x < -\gamma\end{cases},

\gamma, \delta > 0
$$

$P_{t}^{s}$ is the price of stock $s$ at time $t$, $\delta$ is the timeframe for the return calculation, $\gamma$ is the threshold of classifying the return.

#### 2.1 Cleaning the price dataset

In [106]:
import pandas as pd
import numpy as np

data = pd.read_csv("../data/quotes/apple_quotes.csv")

data

,Date,Open,High,Low,Close,Adj Close,Volume
0,02/01/1981,0.154018,0.155134,0.154018,0.154018,0.119849,21660800
1,05/01/1981,0.151228,0.151228,0.150670,0.150670,0.117244,35728000
2,06/01/1981,0.144531,0.144531,0.143973,0.143973,0.112032,45158400
3,07/01/1981,0.138393,0.138393,0.137835,0.137835,0.107256,55686400
4,08/01/1981,0.135603,0.135603,0.135045,0.135045,0.105085,39827200
...,...,...,...,...,...,...,...
10603,23/01/2023,138.119995,143.320007,137.899994,141.110001,141.110001,81760300
10604,24/01/2023,140.309998,143.160004,140.300003,142.529999,142.529999,66435100
10605,25/01/2023,140.889999,142.429993,138.809998,141.860001,141.860001,65799300
10606,26/01/2023,143.169998,144.250000,141.899994,143.960007,143.960007,54105100


In [110]:
# Filter the data to include only dates from 2020 onwards
data = data[data['Date'] >= '2020-01-01']
# Keep only 'Close' and 'Date' columns
data = data[['Date', 'Close']]

# Strip any whitespace from column names
data.columns = data.columns.str.strip()

# Convert 'Close' column to float
data['Close'] = data['Close'].replace('[\$,]', '', regex=True).astype(float)

# Convert 'Date' column to datetime and sort by date
data['Date'] = pd.to_datetime(data['Date'], dayfirst=True)
data = data.sort_values(by='Date')

# Calculate log returns
delta = 1
data['Log_Return'] = np.log(data['Close'] / data['Close'].shift(delta))

# Classification function h(x)
def classify_return(x, gamma):
    if x > gamma:
        return 1  # Positive return
    elif x < -gamma:
        return -1  # Negative return
    else:
        return 0  # Stable return

# Define the threshold for labeling TO OPTIMIZE LATER
gamma = 0.01
data['Return_Label'] = data['Log_Return'].apply(lambda x: classify_return(x, gamma))

# Save the processed dataset
data.to_csv("../data/processed/processed_apple_historical_data.csv", index=False)


Now our quotes dataset includes the log return and looks like this :

In [111]:
data

,Date,Close,Log_Return,Return_Label
9834,2020-01-02,75.087502,NaN,0
9835,2020-01-03,74.357498,-0.009770,0
9836,2020-01-06,74.949997,0.007937,0
9837,2020-01-07,74.597504,-0.004714,0
9838,2020-01-08,75.797501,0.015958,1
...,...,...,...,...
10603,2023-01-23,141.110001,0.023229,1
10604,2023-01-24,142.529999,0.010013,1
10605,2023-01-25,141.860001,-0.004712,0
10606,2023-01-26,143.960007,0.014695,1


#### 2.2 Linking it to the news data
Now that we have a clean dataset to work on, let's merge our sentiment dataset with our returns dataset.

In [126]:
returns_data = pd.read_csv('../data/processed/processed_apple_historical_data.csv')
sentiment_data = pd.read_csv('../data/processed/processed_news_data_with_sent.csv')
sentiment_data.tail(5)

,Unnamed: 0,Date,Stock_Symbol,Article_title,Sentiment
2779,9099,2020-03-10 00:00:00,AAPL,Peloton Shares Tick To Session Low As Hearing Report Apple Working On 'Guided Workout' Fitness App,-1
2780,9105,2020-03-10 00:00:00,AAPL,"UBS Maintains Buy on Apple, Lowers Price Target to $335",1
2781,9115,2020-05-31 00:00:00,AAPL,"Apple CEO Writes To Employees About George Floyd Death, Urges For 'Better, More Just World For Everyone'",0
2782,9135,2020-05-28 00:00:00,AAPL,"Tredje AP-fonden Buys Microsoft Corp, Amazon.com Inc, Apple Inc, Sells iShares U.S. ...",0
2783,9136,2020-05-28 00:00:00,AAPL,"The Zacks Analyst Blog Highlights: Apple, Exxon Mobil, Cisco System and Chevron",0


In [125]:
returns_data.tail(5)

,Date,Close,Log_Return,Return_Label
769,2023-01-23,141.110001,0.023229,1
770,2023-01-24,142.529999,0.010013,1
771,2023-01-25,141.860001,-0.004712,0
772,2023-01-26,143.960007,0.014695,1
773,2023-01-27,145.929993,0.013591,1


I proceed as follos :

-News published after 4 PM is assigned to the next trading day using lambda, 

-Sentiment data is aggregated by trading day using the mean,

-left join with returns_data to retain only rows with both sentiment and return data

-Rows without corresponding sentiment data are dropped using

In [128]:

# 1. Process and check sentiment data and news published after 4PM are assigned to next trading day
sentiment_data['Date'] = pd.to_datetime(sentiment_data['Date'])
sentiment_data['Trading_Date'] = sentiment_data['Date'].apply(
    lambda x: (x + pd.Timedelta(days=1)).date() if x.hour >= 16 else x.date()
)

# 2. Process and check returns data
returns_data['Date'] = pd.to_datetime(returns_data['Date']).dt.date

# 3. Create daily sentiment and check its dates
daily_sentiment = sentiment_data.groupby('Trading_Date')['Sentiment'].mean().reset_index()

# 4. Merge with printed diagnostics
merged_data = pd.merge(
    returns_data,
    daily_sentiment,
    left_on='Date',
    right_on='Trading_Date',
    how='inner'
)

# 6. Check for any overlapping dates
returns_dates = set(returns_data['Date'])
sentiment_dates = set(daily_sentiment['Trading_Date'])
overlapping_dates = returns_dates.intersection(sentiment_dates)

In [129]:
merged_data.head(5)

,Date,Close,Log_Return,Return_Label,Trading_Date,Sentiment
0,2020-03-10,71.334999,0.069546,1,2020-03-10,0.000000
1,2020-03-11,68.857498,-0.035348,-1,2020-03-11,0.142857
2,2020-03-12,62.057499,-0.103978,-1,2020-03-12,0.000000
3,2020-03-13,69.492500,0.113157,1,2020-03-13,0.375000
4,2020-03-16,60.552502,-0.137708,-1,2020-03-16,-0.500000


In [ ]:
merged_data